<a href="https://colab.research.google.com/github/TanishqLambhate/Data-Science-Training/blob/pyspark/Pyspark_Ex_Fitness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col,row_number,rank
spark=SparkSession.builder.appName('Fitness').getOrCreate()
csv_filepath="/content/sample_data/Fitness.csv"
df_fitness=spark.read.csv(csv_filepath,header=True,inferSchema=True)
df_fitness.show()

+-------+----------+-----+--------+-----------+--------------+
|user_id|      date|steps|calories|distance_km|active_minutes|
+-------+----------+-----+--------+-----------+--------------+
|      1|2023-07-01|12000|     500|        8.5|            90|
|      2|2023-07-01| 8000|     350|        5.6|            60|
|      3|2023-07-01|15000|     600|       10.2|           120|
|      1|2023-07-02|11000|     480|        7.9|            85|
|      2|2023-07-02| 9000|     400|        6.2|            70|
|      3|2023-07-02|13000|     520|        9.0|           100|
|      1|2023-07-03|10000|     450|        7.1|            80|
|      2|2023-07-03| 7000|     320|        4.9|            55|
|      3|2023-07-03|16000|     620|       11.0|           130|
+-------+----------+-----+--------+-----------+--------------+



In [10]:
# Exercises:
# 1. Find the Total Steps Taken by Each User
# Calculate the total number of steps taken by each user across all days.

df_steps=df_fitness.groupBy('user_id').sum('steps')
df_steps.show()
# 2. Filter Days Where a User Burned More Than 500 Calories
# Identify all days where a user burned more than 500 calories.
df_caolories=df_fitness.filter(df_fitness['calories']>500)
df_caolories.show()
# 3. Calculate the Average Distance Traveled by Each User
# Calculate the average distance traveled ( distance_km ) by each user
# across all days.
df_avg=df_fitness.groupBy('user_id').avg('distance_km')
df_avg.show()
# 4. Identify the Day with the Maximum Steps for Each User
# For each user, find the day when they took the maximum number of steps.
from pyspark.sql.functions import max
df_max=df_fitness.groupBy('user_id').agg(max('steps').alias('max_steps'))
df_max.show()

+-------+----------+
|user_id|sum(steps)|
+-------+----------+
|      1|     33000|
|      3|     44000|
|      2|     24000|
+-------+----------+

+-------+----------+-----+--------+-----------+--------------+
|user_id|      date|steps|calories|distance_km|active_minutes|
+-------+----------+-----+--------+-----------+--------------+
|      3|2023-07-01|15000|     600|       10.2|           120|
|      3|2023-07-02|13000|     520|        9.0|           100|
|      3|2023-07-03|16000|     620|       11.0|           130|
+-------+----------+-----+--------+-----------+--------------+

+-------+------------------+
|user_id|  avg(distance_km)|
+-------+------------------+
|      1| 7.833333333333333|
|      3|10.066666666666666|
|      2| 5.566666666666667|
+-------+------------------+

+-------+---------+
|user_id|max_steps|
+-------+---------+
|      1|    12000|
|      3|    16000|
|      2|     9000|
+-------+---------+



In [9]:
# 5. Find Users Who Were Active for More Than 100 Minutes on Any Day
# Identify users who had active minutes greater than 100 on any day.
df_active=df_fitness.filter(df_fitness['active_minutes']>100)
df_active.show()

# 6. Calculate the Total Calories Burned per Day
# Group by date and calculate the total number of calories burned by all
df_calories_day=df_fitness.groupBy('date').sum('calories').alias('total_calories_burned')
df_calories_day.show()

# 7. Calculate the Average Steps per Day
# Find the average number of steps taken across all users for each day.
df_avg_steps=df_fitness.groupBy('date').avg('steps').alias('average_steps')
df_avg_steps.show()
# 8. Rank Users by Total Distance Traveled
# Rank the users by their total distance traveled, from highest to lowest.

ranking_by_distance_traveled = df_fitness.groupBy('user_id').agg({'distance_km': 'sum'}).orderBy('sum(distance_km)', ascending=False)
ranking_by_distance_traveled.show()

# 9. Find the Most Active User by Total Active Minutes
# Identify the user with the highest total active minutes across all days.
df_most_active=df_fitness.groupBy('user_id').sum('active_minutes').alias('total_active_minutes')
df_most_active.show()
# 10. Create a New Column for Calories Burned per Kilometer

df_fitness=df_fitness.withColumn('calories_per_km',df_fitness['calories']/df_fitness['distance_km'])
df_fitness.show()

+-------+----------+-----+--------+-----------+--------------+
|user_id|      date|steps|calories|distance_km|active_minutes|
+-------+----------+-----+--------+-----------+--------------+
|      3|2023-07-01|15000|     600|       10.2|           120|
|      3|2023-07-03|16000|     620|       11.0|           130|
+-------+----------+-----+--------+-----------+--------------+

+----------+-------------+
|      date|sum(calories)|
+----------+-------------+
|2023-07-02|         1400|
|2023-07-03|         1390|
|2023-07-01|         1450|
+----------+-------------+

+----------+------------------+
|      date|        avg(steps)|
+----------+------------------+
|2023-07-02|           11000.0|
|2023-07-03|           11000.0|
|2023-07-01|11666.666666666666|
+----------+------------------+

+-------+------------------+
|user_id|  sum(distance_km)|
+-------+------------------+
|      3|              30.2|
|      1|              23.5|
|      2|16.700000000000003|
+-------+------------------+

+-